# Define necessaary libraries

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import torch
import numpy as np
import nltk
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Read data

In [3]:
train_data = load_dataset("cnn_dailymail", '3.0.0', split="train")
train_data = train_data.train_test_split(test_size=0.2)
train_data

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 229690
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 57423
    })
})

# Preprocessing data

In [22]:
checkpoint = "Mprimus/T5-summarize"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [23]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding=True)

    labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [24]:
tokenized_train_data = train_data.map(preprocess_function, batched=True)
tokenized_train_data

Map:   0%|          | 0/229690 [00:00<?, ? examples/s]

Map:   0%|          | 0/57423 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 229690
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 57423
    })
})

# Prepare metric function

In [25]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [26]:
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    b_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    result = {key: value * 100 for key, value in result.items()}
    result["bleu"] = b_result["bleu"]
    
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Train model

In [28]:
training_args = Seq2SeqTrainingArguments(
    output_dir="T5-summarize",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=80,
    per_device_eval_batch_size=80,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
    report_to="none",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data["train"],
    eval_dataset=tokenized_train_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Bleu,Gen Len
1,1.030700,0.924649,24.036400,11.064600,19.981200,22.565900,0.014100,18.984500


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=2872, training_loss=1.0287114836711406, metrics={'train_runtime': 5297.5565, 'train_samples_per_second': 43.358, 'train_steps_per_second': 0.542, 'total_flos': 6.217331676020736e+16, 'train_loss': 1.0287114836711406, 'epoch': 1.0})

In [29]:
# Push model to huggingface
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

'https://huggingface.co/Mprimus/T5-summarize/tree/main/'

# Check model

In [30]:
text = "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [32]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

prefix = "summarize: "

def summarize(text, max_length=128):
    text = prefix + text
    model_inputs = tokenizer.encode(text, return_tensors="pt")
    outputs = model.generate(model_inputs, num_beams=2, max_length=max_length)
    res = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    
    return res[0]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [33]:
summarize(text)

"The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history. No one making under $400,000 per year will pay a penny more in taxes."